In [52]:
# from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
PATH = "data/11.010-1-1.de.pdf"
def load_documents():
    # loader = DirectoryLoader(PATH, glob="*.md")
    loader = UnstructuredPDFLoader(PATH, language="de")
    documents = loader.load()
    return documents

In [53]:
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [55]:
docs = load_documents()
chunks = split_text(docs)

Split 1 documents into 164 chunks.
und kaum eindeutig vorzunehmen. Das Kirchengesetz zählt die «inneren» Angelegenheiten der Kirchen nicht abschliessend auf, sondern umschreibt sie in der Fassung vom 12. September 1995, in Kraft seit dem 1. Juli 1996, wie folgt: «Alles, was sich auf die Wortverkündigung, die Lehre, die Seelsorge,
{'source': 'data/11.010-1-1.de.pdf', 'start_index': 1573}


In [56]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name=r"C:\bjsChatBot\distiluse-base-multilingual-cased-v2"
)
chunk_embeddings = [embeddings.embed_query(chunk.page_content) for chunk in chunks]


No sentence-transformers model found with name C:\bjsChatBot\distiluse-base-multilingual-cased-v2. Creating a new one with mean pooling.


In [ ]:
import faiss
import numpy as np

embedding_matrix = np.array(chunk_embeddings).astype("float32")

d = embedding_matrix.shape[1]

index = faiss.IndexFlatL2(d)  # L2 distance; for cosine, you can normalize first

index.add(embedding_matrix)

faiss.write_index(index, "chunk_index.faiss")

# index = faiss.read_index("chunk_index.faiss")


In [65]:
query = "Pfarrer"
query_embedding = embeddings.embed_query(query)
query_embedding = np.array([query_embedding]).astype("float32")

k = 10  # number of nearest neighbors
distances, indices = index.search(query_embedding, k)

print(indices)    # indices of top 5 chunks
print(distances)  # distances of those chunks


[[119   7 135 125  88  44 110 122  70  13]]
[[1.6658372 2.0091166 2.1625526 2.243872  2.2548225 2.3437304 2.4162385
  2.428784  2.4334397 2.4753718]]


In [66]:
# Suppose these are the results from FAISS
indices = [119, 7, 135, 125,  88]
distances = [1.6658372, 2.0091166, 2.1625526, 2.243872,  2.2548225, 2.3437304, 2.4162385,
  2.428784,  2.4334397, 2.4753718]

# Map indices back to your original text chunks and metadata
for idx, dist in zip(indices, distances):
    chunk = chunks[idx]  # get the Document object
    print(f"Index: {idx}, Distance: {dist}")
    print("Text:")
    print(chunk.page_content)
    print("Metadata:")
    print(chunk.metadata)
    print("-" * 50)


Index: 119, Distance: 1.6658372
Text:
Geboten der Leh- re des Heils in allen Stücken Zeugnis zu geben und die Aufgaben seines Amtes treu und gewissenhaft zu erfüllen.
Metadata:
{'source': 'data/11.010-1-1.de.pdf', 'start_index': 24379}
--------------------------------------------------
Index: 7, Distance: 2.0091166
Text:
stimmberechtigten Glieder der Landeskir- che haben obligatorisch über die Schaffung oder Änderung der Kirchenver- fassung zu entscheiden.
Metadata:
{'source': 'data/11.010-1-1.de.pdf', 'start_index': 1139}
--------------------------------------------------
Index: 135, Distance: 2.1625526
Text:
4 Der Geldhaushalt der Kirche

Art. B4

Lukas 16,10

1 Wer im Geringsten treu ist, der ist auch im Grossen treu; und wer im Ge- ringsten unrecht ist, der ist auch im Grossen unrecht.

Art. 36

Geldmittel der Kirche
Metadata:
{'source': 'data/11.010-1-1.de.pdf', 'start_index': 27499}
--------------------------------------------------
Index: 125, Distance: 2.243872
Text:
empfehlen.